**Import Libraries**

In [18]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

**MOUNT GOOGLE DRIVE**

In [19]:
from google.colab import drive
drive.mount('/content/drive')

import os
project_folder = '/content/drive/MyDrive/FraudDetection'  # Sesuaikan dengan folder kamu
os.chdir(project_folder)

print(f"Working Directory: {os.getcwd()}")
print("\n✅ Google Drive mounted!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Working Directory: /content/drive/MyDrive/FraudDetection

✅ Google Drive mounted!


**LOAD DATA & COLUMN INFO**

In [21]:
print("\n" + "=" * 60)
print("LOADING DATA")
print("=" * 60)

# Load training data
train_df = pd.read_csv('/content/drive/MyDrive/datasets/train_transaction.csv')
print(f"Original Dataset Shape: {train_df.shape}")
print(f"Memory Usage: {train_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Load column info from Notebook 1
with open('results/01_column_info.pkl', 'rb') as f:
    column_info = pickle.load(f)

numeric_cols = column_info['numeric_columns']
categorical_cols = column_info['categorical_columns']

print(f"\nNumeric Columns: {len(numeric_cols)}")
print(f"Categorical Columns: {len(categorical_cols)}")


LOADING DATA
Original Dataset Shape: (590540, 394)
Memory Usage: 2062.07 MB

Numeric Columns: 379
Categorical Columns: 14


**REMOVE DUPLICATES**

In [22]:
print("\n" + "=" * 60)
print("STEP 1: REMOVING DUPLICATES")
print("=" * 60)

initial_rows = len(train_df)
train_df = train_df.drop_duplicates()
removed_duplicates = initial_rows - len(train_df)

print(f"Removed {removed_duplicates} duplicate rows")
print(f"Remaining rows: {len(train_df):,}")


STEP 1: REMOVING DUPLICATES
Removed 0 duplicate rows
Remaining rows: 590,540


**DROP COLUMNS WITH >50% MISSING VALUES**

In [23]:
print("\n" + "=" * 60)
print("STEP 2: DROPPING COLUMNS WITH >50% MISSING VALUES")
print("=" * 60)

# Calculate missing percentage
missing_pct = (train_df.isnull().sum() / len(train_df)) * 100
cols_to_drop = missing_pct[missing_pct > 50].index.tolist()

print(f"Columns to drop ({len(cols_to_drop)}):")
if len(cols_to_drop) > 0:
    for col in cols_to_drop[:20]:  # Show first 20
        print(f"  - {col}: {missing_pct[col]:.2f}% missing")
    if len(cols_to_drop) > 20:
        print(f"  ... and {len(cols_to_drop) - 20} more")

    train_df = train_df.drop(columns=cols_to_drop)

    # Update column lists
    numeric_cols = [col for col in numeric_cols if col not in cols_to_drop]
    categorical_cols = [col for col in categorical_cols if col not in cols_to_drop]

    print(f"\n✓ Dropped {len(cols_to_drop)} columns")
else:
    print("✓ No columns with >50% missing values")

print(f"Current shape: {train_df.shape}")

# Save dropped columns list
with open('results/02_dropped_columns.pkl', 'wb') as f:
    pickle.dump(cols_to_drop, f)


STEP 2: DROPPING COLUMNS WITH >50% MISSING VALUES
Columns to drop (174):
  - dist1: 59.65% missing
  - dist2: 93.63% missing
  - R_emaildomain: 76.75% missing
  - D5: 52.47% missing
  - D6: 87.61% missing
  - D7: 93.41% missing
  - D8: 87.31% missing
  - D9: 87.31% missing
  - D12: 89.04% missing
  - D13: 89.51% missing
  - D14: 89.47% missing
  - M5: 59.35% missing
  - M7: 58.64% missing
  - M8: 58.63% missing
  - M9: 58.63% missing
  - V138: 86.12% missing
  - V139: 86.12% missing
  - V140: 86.12% missing
  - V141: 86.12% missing
  - V142: 86.12% missing
  ... and 154 more

✓ Dropped 174 columns
Current shape: (590540, 220)


**IMPUTE MISSING VALUES**

In [24]:
print("\n" + "=" * 60)
print("STEP 3: IMPUTING MISSING VALUES")
print("=" * 60)

# Numeric columns: Impute with median
imputation_stats = {}

print("\nImputing Numeric Columns with Median...")
for col in numeric_cols:
    if train_df[col].isnull().sum() > 0:
        median_value = train_df[col].median()
        missing_count = train_df[col].isnull().sum()
        train_df[col].fillna(median_value, inplace=True)
        imputation_stats[col] = {'type': 'numeric', 'method': 'median',
                                  'value': median_value, 'filled': missing_count}
        print(f"  ✓ {col}: filled {missing_count} values with median {median_value:.2f}")

print(f"\nImputed {len([k for k, v in imputation_stats.items() if v['type'] == 'numeric'])} numeric columns")

# Categorical columns: Impute with mode or 'Unknown'
print("\nImputing Categorical Columns with Mode...")
for col in categorical_cols:
    if train_df[col].isnull().sum() > 0:
        missing_count = train_df[col].isnull().sum()
        if train_df[col].mode().shape[0] > 0:
            mode_value = train_df[col].mode()[0]
        else:
            mode_value = 'Unknown'
        train_df[col].fillna(mode_value, inplace=True)
        imputation_stats[col] = {'type': 'categorical', 'method': 'mode',
                                  'value': mode_value, 'filled': missing_count}
        print(f"  ✓ {col}: filled {missing_count} values with '{mode_value}'")

print(f"\nImputed {len([k for k, v in imputation_stats.items() if v['type'] == 'categorical'])} categorical columns")

# Save imputation statistics
with open('results/02_imputation_stats.pkl', 'wb') as f:
    pickle.dump(imputation_stats, f)

# Verify no missing values remain
remaining_missing = train_df.isnull().sum().sum()
print(f"\n✓ Total remaining missing values: {remaining_missing}")


STEP 3: IMPUTING MISSING VALUES

Imputing Numeric Columns with Median...
  ✓ card2: filled 8933 values with median 361.00
  ✓ card3: filled 1565 values with median 150.00
  ✓ card5: filled 4259 values with median 226.00
  ✓ addr1: filled 65706 values with median 299.00
  ✓ addr2: filled 65706 values with median 87.00
  ✓ D1: filled 1269 values with median 3.00
  ✓ D2: filled 280797 values with median 97.00
  ✓ D3: filled 262878 values with median 8.00
  ✓ D4: filled 168922 values with median 26.00
  ✓ D10: filled 76022 values with median 15.00
  ✓ D11: filled 279287 values with median 43.00
  ✓ D15: filled 89113 values with median 52.00
  ✓ V1: filled 279287 values with median 1.00
  ✓ V2: filled 279287 values with median 1.00
  ✓ V3: filled 279287 values with median 1.00
  ✓ V4: filled 279287 values with median 1.00
  ✓ V5: filled 279287 values with median 1.00
  ✓ V6: filled 279287 values with median 1.00
  ✓ V7: filled 279287 values with median 1.00
  ✓ V8: filled 279287 values wit

**HANDLE OUTLIERS (CAPPING METHOD)**

In [25]:
print("\n" + "=" * 60)
print("STEP 4: HANDLING OUTLIERS (IQR CAPPING)")
print("=" * 60)

outlier_caps = {}

print("\nCapping outliers for numeric columns...")
for col in numeric_cols[:10]:  # Process first 10 to save time, adjust as needed
    Q1 = train_df[col].quantile(0.25)
    Q3 = train_df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Count outliers before capping
    outliers_count = ((train_df[col] < lower_bound) | (train_df[col] > upper_bound)).sum()

    if outliers_count > 0:
        # Cap outliers
        train_df[col] = train_df[col].clip(lower=lower_bound, upper=upper_bound)
        outlier_caps[col] = {'lower': lower_bound, 'upper': upper_bound, 'capped': outliers_count}
        print(f"  ✓ {col}: capped {outliers_count} outliers")

print(f"\n✓ Processed outliers for {len(outlier_caps)} columns")

# Save outlier caps for test data
with open('results/02_outlier_caps.pkl', 'wb') as f:
    pickle.dump(outlier_caps, f)


STEP 4: HANDLING OUTLIERS (IQR CAPPING)

Capping outliers for numeric columns...
  ✓ TransactionAmt: capped 66482 outliers
  ✓ card3: capped 67688 outliers
  ✓ addr1: capped 8807 outliers
  ✓ addr2: capped 4353 outliers
  ✓ C1: capped 59535 outliers

✓ Processed outliers for 5 columns


**OPTIMIZE DATA TYPES (MEMORY REDUCTION)**

In [26]:
print("\n" + "=" * 60)
print("STEP 5: OPTIMIZING DATA TYPES")
print("=" * 60)

memory_before = train_df.memory_usage(deep=True).sum() / 1024**2

# Optimize numeric columns
for col in numeric_cols:
    col_type = train_df[col].dtype

    if col_type == 'int64':
        c_min = train_df[col].min()
        c_max = train_df[col].max()
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
            train_df[col] = train_df[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
            train_df[col] = train_df[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
            train_df[col] = train_df[col].astype(np.int32)

    elif col_type == 'float64':
        train_df[col] = train_df[col].astype(np.float32)

# Optimize categorical columns with limited unique values
for col in categorical_cols:
    num_unique_values = train_df[col].nunique()
    num_total_values = len(train_df[col])
    if num_unique_values / num_total_values < 0.5:  # If less than 50% unique
        train_df[col] = train_df[col].astype('category')

memory_after = train_df.memory_usage(deep=True).sum() / 1024**2
memory_saved = memory_before - memory_after
memory_reduction_pct = (memory_saved / memory_before) * 100

print(f"Memory before: {memory_before:.2f} MB")
print(f"Memory after: {memory_after:.2f} MB")
print(f"Memory saved: {memory_saved:.2f} MB ({memory_reduction_pct:.2f}%)")


STEP 5: OPTIMIZING DATA TYPES
Memory before: 1214.67 MB
Memory after: 481.53 MB
Memory saved: 733.14 MB (60.36%)


**SAVE CLEANED DATA**

In [27]:
print("\n" + "=" * 60)
print("SAVING CLEANED DATA")
print("=" * 60)

# Save cleaned data
train_df.to_csv('data/cleaned/train_cleaned.csv', index=False)
print(f"💾 Saved: data/cleaned/train_cleaned.csv")
print(f"   Shape: {train_df.shape}")
print(f"   Size: {os.path.getsize('data/cleaned/train_cleaned.csv') / 1024**2:.2f} MB")

# Update and save column info
updated_column_info = {
    'numeric_columns': [col for col in numeric_cols if col in train_df.columns],
    'categorical_columns': [col for col in categorical_cols if col in train_df.columns],
    'all_columns': train_df.columns.tolist(),
    'target_column': 'isFraud'
}

with open('results/02_updated_column_info.pkl', 'wb') as f:
    pickle.dump(updated_column_info, f)

print("💾 Saved: results/02_updated_column_info.pkl")


SAVING CLEANED DATA
💾 Saved: data/cleaned/train_cleaned.csv
   Shape: (590540, 220)
   Size: 515.95 MB
💾 Saved: results/02_updated_column_info.pkl


**PREPROCESSING SUMMARY**

In [28]:
print("\n" + "=" * 60)
print("PREPROCESSING SUMMARY")
print("=" * 60)

summary = f"""
PREPROCESSING COMPLETED SUCCESSFULLY!

Original Shape: {initial_rows} rows × {len(column_info['all_columns'])} columns
Final Shape: {train_df.shape[0]} rows × {train_df.shape[1]} columns

Steps Completed:
1. ✓ Removed {removed_duplicates} duplicate rows
2. ✓ Dropped {len(cols_to_drop)} columns with >50% missing values
3. ✓ Imputed {len(imputation_stats)} columns with missing values
4. ✓ Capped outliers in {len(outlier_caps)} numeric columns
5. ✓ Optimized data types (saved {memory_saved:.2f} MB)

Final Dataset:
- Numeric columns: {len(updated_column_info['numeric_columns'])}
- Categorical columns: {len(updated_column_info['categorical_columns'])}
- Total columns: {len(updated_column_info['all_columns'])}
- Memory usage: {memory_after:.2f} MB
- No missing values: {train_df.isnull().sum().sum() == 0}

Next Step: Proceed to Notebook 3 for Feature Engineering
"""

print(summary)

# Save summary
with open('results/02_preprocessing_summary.txt', 'w') as f:
    f.write(summary)

print("💾 Saved: results/02_preprocessing_summary.txt")


PREPROCESSING SUMMARY

PREPROCESSING COMPLETED SUCCESSFULLY!

Original Shape: 590540 rows × 394 columns
Final Shape: 590540 rows × 220 columns

Steps Completed:
1. ✓ Removed 0 duplicate rows
2. ✓ Dropped 174 columns with >50% missing values
3. ✓ Imputed 200 columns with missing values
4. ✓ Capped outliers in 5 numeric columns
5. ✓ Optimized data types (saved 733.14 MB)

Final Dataset:
- Numeric columns: 210
- Categorical columns: 9
- Total columns: 220
- Memory usage: 481.53 MB
- No missing values: True

Next Step: Proceed to Notebook 3 for Feature Engineering

💾 Saved: results/02_preprocessing_summary.txt


**MEMORY CLEANUP**

In [29]:
print("\n" + "=" * 60)
print("MEMORY CLEANUP")
print("=" * 60)

del column_info, imputation_stats, outlier_caps
gc.collect()

print("✓ Memory cleaned")
print("\n" + "=" * 60)
print("NOTEBOOK 2 COMPLETE!")
print("=" * 60)


MEMORY CLEANUP
✓ Memory cleaned

NOTEBOOK 2 COMPLETE!
